In [16]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 23: Opening the Turing Lock ---</h2><p>Little Jane Marie just got her very first computer for Christmas from some <span title="Definitely not Wintermute.">unknown benefactor</span>.  It comes with instructions and an example program, but the computer itself seems to be malfunctioning.  She's curious what the program does, and would like you to help her run it.</p>
<p>The manual explains that the computer supports two <a href="https://en.wikipedia.org/wiki/Processor_register">registers</a> and six <a href="https://en.wikipedia.org/wiki/Instruction_set">instructions</a> (truly, it goes on to remind the reader, a state-of-the-art technology). The registers are named <code>a</code> and <code>b</code>, can hold any <a href="https://en.wikipedia.org/wiki/Natural_number">non-negative integer</a>, and begin with a value of <code>0</code>.  The instructions are as follows:</p>
<ul>
<li><code>hlf r</code> sets register <code>r</code> to <em>half</em> its current value, then continues with the next instruction.</li>
<li><code>tpl r</code> sets register <code>r</code> to <em>triple</em> its current value, then continues with the next instruction.</li>
<li><code>inc r</code> <em>increments</em> register <code>r</code>, adding <code>1</code> to it, then continues with the next instruction.</li>
<li><code>jmp offset</code> is a <em>jump</em>; it continues with the instruction <code>offset</code> away <em>relative to itself</em>.</li>
<li><code>jie r, offset</code> is like <code>jmp</code>, but only jumps if register <code>r</code> is <em>even</em> ("jump if even").</li>
<li><code>jio r, offset</code> is like <code>jmp</code>, but only jumps if register <code>r</code> is <code>1</code> ("jump if <em>one</em>", not odd).</li>
</ul>
<p>All three jump instructions work with an <em>offset</em> relative to that instruction.  The offset is always written with a prefix <code>+</code> or <code>-</code> to indicate the direction of the jump (forward or backward, respectively).  For example, <code>jmp +1</code> would simply continue with the next instruction, while <code>jmp +0</code> would continuously jump back to itself forever.</p>
<p>The program exits when it tries to run an instruction beyond the ones defined.</p>
<p>For example, this program sets <code>a</code> to <code>2</code>, because the <code>jio</code> instruction causes it to skip the <code>tpl</code> instruction:</p>
<pre><code>inc a
jio a, +2
tpl a
inc a
</code></pre>
<p>What is <em>the value in register <code>b</code></em> when the program in your puzzle input is finished executing?</p>
</article>


In [106]:
programm = """
inc a
jio a, +2
tpl a
inc a
"""


@dataclass
class Instruction:
    instruction: str
    register: str


@dataclass
class Jump:
    instruction: str
    offset: int


@dataclass
class ConditionalJump:
    instruction: str
    register: str
    offset: int


class TurningMachine:
    def __init__(self, programm: str) -> None:
        self.registers = {"a": 0, "b": 0}
        self.programm = self._parse(programm)

    def _parse(self, programm: str):
        return [self._parse_step(step) for step in programm.strip().splitlines()]

    def _parse_step(self, step: str) -> Instruction | Jump | ConditionalJump:
        match (step[:3]):
            case "hlf" | "tpl" | "tpl" | "inc":
                return Instruction(step[:3], step[4:])
            case "jmp":
                return Jump(step[:3], int(step[4:]))
            case "jie" | "jio":
                return ConditionalJump(step[:3], step[4:5], int(step[6:]))
            case _:
                return Instruction(step[:3], step[4:5])

    def process(self):
        counter = 0
        while 0 <= counter < len(self.programm):
            match self.programm[counter]:
                case Instruction(instruction="hlf", register=register):
                    # hlf r sets register r to half its current value,
                    self.registers[register] //= 2
                    # then continues with the next instruction.
                    counter += 1
                case Instruction(instruction="tpl", register=register):
                    # tpl r sets register r to triple its current value,
                    self.registers[register] *= 3
                    # then continues with the next instruction.
                    counter += 1
                case Instruction(instruction="inc", register=register):
                    # inc r increments register r, adding 1 to it,
                    self.registers[register] += 1
                    # then continues with the next instruction.
                    counter += 1
                case Jump(instruction="jmp", offset=offset):
                    # jmp offset is a jump;
                    # it continues with the instruction offset away relative to itself.
                    counter += int(offset)
                case ConditionalJump(
                    instruction="jie", register=register, offset=offset
                ):
                    # jie r, offset is like jmp,
                    # but only jumps if register r is even ("jump if even").
                    if self.registers[register] % 2 == 0:
                        counter += offset
                    else:
                        counter += 1
                case ConditionalJump(
                    instruction="jio", register=register, offset=offset
                ):
                    # jio r, offset is like jmp,
                    # but only jumps if register r is 1 ("jump if one", not odd).
                    if self.registers[register] == 1:
                        counter += offset
                    else:
                        counter += 1
                case _:
                    raise ValueError(f"Unknown instruction {self.programm[counter]}!")

    def __repr__(self) -> str:
        return f"TurningMachine{self.registers}"


tm = TurningMachine(programm)
tm.process()
assert tm.registers == {"a": 2, "b": 0}

In [107]:
programm = """
inc a
inc a
tpl a
"""
tm = TurningMachine(programm)
tm.process()
assert tm.registers == {"a": 6, "b": 0}

In [108]:
programm = """
inc a
jmp +2
inc a
tpl a
"""
tm = TurningMachine(programm)
tm.process()
assert tm.registers == {"a": 3, "b": 0}

In [109]:
programm = """
inc a
jie a -1
inc a
tpl a
"""
tm = TurningMachine(programm)
tm.process()
assert tm.registers == {"a": 6, "b": 0}

In [110]:
programm = """
inc a
jio a -1
inc a
tpl a
"""
tm = TurningMachine(programm)
tm.process()
assert tm.registers == {"a": 9, "b": 0}

In [111]:
programm = """
inc a
jie a -1
inc a
tpl a
"""
tm = TurningMachine(programm)
tm.process()
assert tm.registers == {"a": 6, "b": 0}

In [112]:
programm = """
inc a
jie a -1
inc a
tpl a
hlf a
"""
tm = TurningMachine(programm)
tm.process()
assert tm.registers == {"a": 3, "b": 0}

In [113]:
with open("../input/day23.txt") as f:
    tmf = TurningMachine(f.read())
tmf.process()
tmf

TurningMachine{'a': 1, 'b': 307}

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>307</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The unknown benefactor is <em>very</em> thankful for releasi-- er, helping little Jane Marie with her computer.  Definitely not to distract you, what is the value in register <code>b</code> after the program is finished executing if register <code>a</code> starts as <code>1</code> instead?</p>
</article>

</main>


In [114]:
with open("../input/day23.txt") as f:
    tmf = TurningMachine(f.read())
tmf.registers["a"] = 1
tmf.process()
tmf

TurningMachine{'a': 1, 'b': 160}

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>160</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="23/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Opening+the+Turing+Lock%22+%2D+Day+23+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F23&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Opening+the+Turing+Lock%22+%2D+Day+23+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F23';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
